# Word2Vec Skipgram impl. in PyTorch

["Word2Vec in Pytorch - Continuous Bag of Words and Skipgrams: Pytorch implementation"](https://srijithr.gitlab.io/post/word2vec/)  
Srijith Rajamohan (2018-09-09)

which was adapted from <https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#an-example-n-gram-language-modeling>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import urllib.request

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize

import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
def get_key (word_id):
    for key, val in word_to_ix.items():
        if val == word_id:
            print(key)


def cluster_embeddings (filename, nclusters):
    X = np.load(filename)
    kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(X)
    center = kmeans.cluster_centers_
    distances = euclidean_distances(X, center)

    for i in np.arange(0, distances.shape[1]):
        word_id = np.argmin(distances[:, i])
        print(word_id)
        get_key(word_id)


def read_data (file_path):
    tokenizer = RegexpTokenizer(r'\w+')
    data = urllib.request.urlopen(file_path)
    data = data.read().decode('utf8')
    
    tokenized_data = word_tokenize(data)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.', ',', ':', ';', '(', ')', '#', '--', '...', '"'])
    
    cleaned_words = [ i for i in tokenized_data if i not in stop_words ]
    return(cleaned_words)

In [3]:
CONTEXT_SIZE = 3
EMBEDDING_DIM = 10

torch.manual_seed(1)

test_sentence = """Empathy for the poor may not come easily to people who never experienced it. They may blame the victims and insist their predicament can be overcome through determination and hard work.
But they may not realize that extreme poverty can be psychologically and physically incapacitating — a perpetual cycle of bad diets, health care and education exacerbated by the shaming and self-fulfilling prophecies that define it in the public imagination.
Gordon Parks — perhaps more than any artist — saw poverty as “the most savage of all human afflictions” and realized the power of empathy to help us understand it. It was neither an abstract problem nor political symbol, but something he endured growing up destitute in rural Kansas and having spent years documenting poverty throughout the world, including the United States.
That sensitivity informed “Freedom’s Fearful Foe: Poverty,” his celebrated photo essay published in Life magazine in June 1961. He took readers into the lives of a Brazilian boy, Flavio da Silva, and his family, who lived in the ramshackle Catacumba favela in the hills outside Rio de Janeiro. These stark photographs are the subject of a new book, “Gordon Parks: The Flavio Story” (Steidl/The Gordon Parks Foundation), which accompanies a traveling exhibition co-organized by the Ryerson Image Centre in Toronto, where it opens this week, and the J. Paul Getty Museum. Edited with texts by the exhibition’s co-curators, Paul Roth and Amanda Maddox, the book also includes a recent interview with Mr. da Silva and essays by Beatriz Jaguaribe, Maria Alice Rezende de Carvalho and Sérgio Burgi.
""".split()

In [4]:
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

#test_sentence = read_data('https://www.gutenberg.org/files/57884/57884-0.txt')

ngrams = []

for i in range(len(test_sentence) - CONTEXT_SIZE):
    tup = [test_sentence[j] for j in np.arange(i + 1 , i + CONTEXT_SIZE + 1) ]
    ngrams.append((test_sentence[i],tup))

# print the first 3, just so you can see what they look like
print(ngrams)

[('Empathy', ['for', 'the', 'poor']), ('for', ['the', 'poor', 'may']), ('the', ['poor', 'may', 'not']), ('poor', ['may', 'not', 'come']), ('may', ['not', 'come', 'easily']), ('not', ['come', 'easily', 'to']), ('come', ['easily', 'to', 'people']), ('easily', ['to', 'people', 'who']), ('to', ['people', 'who', 'never']), ('people', ['who', 'never', 'experienced']), ('who', ['never', 'experienced', 'it.']), ('never', ['experienced', 'it.', 'They']), ('experienced', ['it.', 'They', 'may']), ('it.', ['They', 'may', 'blame']), ('They', ['may', 'blame', 'the']), ('may', ['blame', 'the', 'victims']), ('blame', ['the', 'victims', 'and']), ('the', ['victims', 'and', 'insist']), ('victims', ['and', 'insist', 'their']), ('and', ['insist', 'their', 'predicament']), ('insist', ['their', 'predicament', 'can']), ('their', ['predicament', 'can', 'be']), ('predicament', ['can', 'be', 'overcome']), ('can', ['be', 'overcome', 'through']), ('be', ['overcome', 'through', 'determination']), ('overcome', ['thr

In [5]:
vocab = set(test_sentence)
print("Length of vocabulary", len(vocab))

word_to_ix = {word: i for i, word in enumerate(vocab)}
word_to_ix

Length of vocabulary 195


{'hills': 0,
 'imagination.': 1,
 'bad': 2,
 'any': 3,
 'Flavio': 4,
 'Museum.': 5,
 'Life': 6,
 'blame': 7,
 'Image': 8,
 'cycle': 9,
 'family,': 10,
 '“Gordon': 11,
 'most': 12,
 'savage': 13,
 'artist': 14,
 'lived': 15,
 'are': 16,
 'Mr.': 17,
 'Foe:': 18,
 'It': 19,
 'psychologically': 20,
 'Jaguaribe,': 21,
 'endured': 22,
 'who': 23,
 'He': 24,
 'nor': 25,
 'opens': 26,
 'Rezende': 27,
 'may': 28,
 'he': 29,
 'Fearful': 30,
 'care': 31,
 'interview': 32,
 'saw': 33,
 'help': 34,
 'can': 35,
 'perhaps': 36,
 'Janeiro.': 37,
 'Paul': 38,
 'Maddox,': 39,
 'than': 40,
 'poverty': 41,
 'self-fulfilling': 42,
 'rural': 43,
 'but': 44,
 'Brazilian': 45,
 'Maria': 46,
 'insist': 47,
 'extreme': 48,
 'June': 49,
 'Silva': 50,
 'Getty': 51,
 'the': 52,
 'easily': 53,
 'Empathy': 54,
 'come': 55,
 'which': 56,
 'an': 57,
 'photographs': 58,
 'shaming': 59,
 'of': 60,
 '“the': 61,
 'afflictions”': 62,
 'perpetual': 63,
 'exhibition’s': 64,
 'up': 65,
 'Parks:': 66,
 'Foundation),': 67,
 'by

In [6]:
class SkipgramModeler (nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(SkipgramModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, context_size * vocab_size)
        #self.parameters['context_size'] = context_size

        
    def forward (self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # -1 implies size inferred for that index from the size of the data
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        
        out1 = F.relu(self.linear1(embeds)) # output of first layer
        out2 = self.linear2(out1)           # output of second layer
        #print(embeds)
        
        log_probs = F.log_softmax(out2, dim=1).view(CONTEXT_SIZE,-1)
        return log_probs

    
    def predict (self, input):
        context_idxs = torch.tensor([word_to_ix[input]], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        indices = [res_ind[i][0] for i in np.arange(0, 3)]
        
        for arg in indices:
            print( [ (key, val) for key,val in word_to_ix.items() if val == arg ])


    def freeze_layer (self, layer):
        for name, child in model.named_children():
            print(name, child)
            
            if name == layer:
                for names, params in child.named_parameters():
                    print(names, params)
                    print(params.size())
                    params.requires_grad = False

                    
    def print_layer_parameters (self):
        for name, child in model.named_children():
                print(name, child)
                
                for names,params in child.named_parameters():
                    print(names, params)
                    print(params.size())

                    
    def write_embedding_to_file (self, filename):
        for i in self.embeddings.parameters():
            weights = i.data.numpy()
            
        np.save(filename, weights)

In [7]:
from tqdm import tqdm

losses = []
loss_function = nn.NLLLoss()

model = SkipgramModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Freeze embedding layer
#model.freeze_layer('embeddings')

for epoch in tqdm(range(550)):
    total_loss = 0
    
    #------- Embedding layers are trained as well here ----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#

    model.predict('psychologically')

    for context, target in ngrams:
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        #print(context,target)

        context_idxs = torch.tensor([word_to_ix[context]], dtype=torch.long)
        #print("Context id",context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        #print(log_probs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        target_list = torch.tensor([word_to_ix[w] for w in target], dtype=torch.long)
        loss = loss_function(log_probs, target_list)
        #print(loss)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
        
    print(total_loss)
    losses.append(total_loss)

  0%|          | 0/550 [00:00<?, ?it/s]

[('insist', 47)]
[('Janeiro.', 37)]
[('determination', 167)]


  0%|          | 1/550 [00:00<01:43,  5.29it/s]

1639.6772470474243
[('insist', 47)]
[('Janeiro.', 37)]
[('determination', 167)]


  0%|          | 2/550 [00:00<01:38,  5.58it/s]

...

514.367312669754
[('and', 72)]
[('physically', 117)]
[('incapacitating', 99)]
513.8650171756744
[('and', 72)]
[('physically', 117)]
[('incapacitating', 99)]


100%|█████████▉| 548/550 [01:24<00:00,  6.88it/s]

513.3660607337952
[('and', 72)]
[('physically', 117)]
[('incapacitating', 99)]
512.8708571195602
[('and', 72)]
[('physically', 117)]
[('incapacitating', 99)]


100%|██████████| 550/550 [01:24<00:00,  6.49it/s]

512.3788785934448
[('and', 72)]
[('physically', 117)]
[('incapacitating', 99)]
511.89018642902374


In [8]:
#print(losses)  # The loss decreased every iteration over the training data!

#Print the model layer parameters
#model.print_layer_parameters()

#Predict the next word given n context words
model.predict('psychologically')
model.write_embedding_to_file('embeddings_skipgrams.npy')

cluster_embeddings('embeddings_skipgrams.npy', 5)

[('and', 72)]
[('physically', 117)]
[('incapacitating', 99)]
103
more
193
where
116
experienced
56
which
88
informed
